In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.18.0


# Tensor operations

In [3]:
# let's create our first tensor
# this is a constant tensor, meaning it is immutable, the values inside it
# may not be changed in place
tensor = tf.constant([[1,2],[3,4]])
print(tensor)

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)


In [4]:
# Ajouter un scalaire à un tensor
print(tensor + 5)

# Ajout d'une matrice explicitement
print(tensor + tf.constant([[5, 5], [5, 5]]))

tf.Tensor(
[[6 7]
 [8 9]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[6 7]
 [8 9]], shape=(2, 2), dtype=int32)


In [5]:
# Définir un second tensor
tensor2 = tf.constant([[5, 6], [7, 8]])

# Ajouter les deux tensors
print(tensor + tensor2)

tf.Tensor(
[[ 6  8]
 [10 12]], shape=(2, 2), dtype=int32)


In [6]:
# Définir un tensor de forme différente
tensor3 = tf.constant([1, 2])

# Ajouter les deux tensors
print(tensor + tensor3)

# Équivalent explicite avec broadcasting
print(tensor + tf.constant([[1, 2], [1, 2]]))

tf.Tensor(
[[2 4]
 [4 6]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[2 4]
 [4 6]], shape=(2, 2), dtype=int32)


In [7]:
# Définir un tensor de forme différente
tensor4 = tf.constant([[1], [2]])

# Ajouter les deux tensors
print(tensor + tensor4)

# Équivalent explicite avec broadcasting
print(tensor + tf.constant([[1, 1], [2, 2]]))

tf.Tensor(
[[2 3]
 [5 6]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[2 3]
 [5 6]], shape=(2, 2), dtype=int32)


In [8]:
# Multiplication by a scalar
print(tensor * 4)

tf.Tensor(
[[ 4  8]
 [12 16]], shape=(2, 2), dtype=int32)


In [9]:
# Pointwise multiplication by a tensor of same shape
print(tensor * tensor2)

tf.Tensor(
[[ 5 12]
 [21 32]], shape=(2, 2), dtype=int32)


In [10]:
# Pointwise multiplication of tensors of different shapes
print(tensor * tensor3)
print(tensor * tensor4)

tf.Tensor(
[[1 4]
 [3 8]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[1 2]
 [6 8]], shape=(2, 2), dtype=int32)


In [11]:
# Matrix multiplication of tensors
print(tf.matmul(tensor, tensor2))
print(tf.matmul(tensor, tensor4))

tf.Tensor(
[[19 22]
 [43 50]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[ 5]
 [11]], shape=(2, 1), dtype=int32)


In [12]:
# Définir une variable tensor
variable_tensor = tf.Variable([[1, 2], [3, 4]])

# Ajouter une matrice à la variable
variable_tensor.assign_add([[1, 1], [1, 1]])
print(variable_tensor)

# Soustraire une matrice à la variable
variable_tensor.assign_sub([[2, 2], [2, 2]])
print(variable_tensor)

# Réassigner complètement la valeur de la variable
variable_tensor.assign([[1, 2], [3, 4]])
print(variable_tensor)

<tf.Variable 'Variable:0' shape=(2, 2) dtype=int32, numpy=
array([[2, 3],
       [4, 5]], dtype=int32)>
<tf.Variable 'Variable:0' shape=(2, 2) dtype=int32, numpy=
array([[0, 1],
       [2, 3]], dtype=int32)>
<tf.Variable 'Variable:0' shape=(2, 2) dtype=int32, numpy=
array([[1, 2],
       [3, 4]], dtype=int32)>


In [13]:
tensor.numpy()

array([[1, 2],
       [3, 4]], dtype=int32)

In [14]:
tensor.shape # gives the shape of the tensor

TensorShape([2, 2])

In [15]:
tf.reshape(tensor, [-1,1]) # reshapes the tensor

<tf.Tensor: shape=(4, 1), dtype=int32, numpy=
array([[1],
       [2],
       [3],
       [4]], dtype=int32)>

In [16]:
from sklearn.datasets import load_iris 
iris = load_iris() # loding the iris dataset
data = iris.data # storing data in a separate object
target = iris.target # storin the target in a separate object

In [17]:
# To train deep learning models, we will use batch gradient descent
# Therefore we are going to form batch datasets with tensorflow

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data,target)

# to form a tensor dataset we will use a function call from_tensor_slices
# that converts tuples of arrays into tensor datasets
train = tf.data.Dataset.from_tensor_slices((X_train,y_train))
test = tf.data.Dataset.from_tensor_slices((X_test,y_test))

# to extract a tensor from these objects we can use two different techniques :
x, y = next(iter(train)) # iter turns train into an iterator and next picks the next element of train
print('x:',x)
print('y:',y)

for x, y in train.take(1): #take will give you the first n tensors in the dataset
  print('x:',x)
  print('y:',y)

x: tf.Tensor([6.5 3.  5.8 2.2], shape=(4,), dtype=float64)
y: tf.Tensor(2, shape=(), dtype=int64)
x: tf.Tensor([6.5 3.  5.8 2.2], shape=(4,), dtype=float64)
y: tf.Tensor(2, shape=(), dtype=int64)


2024-12-15 18:21:30.020023: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [18]:
# let's see the type of object obtained for train
train
# it's a TensorSliceDataset that contains tuples of tensors of respective shapes
# (4,) (meaning 4 columns) and () (meaning it's a scalar)

<_TensorSliceDataset element_spec=(TensorSpec(shape=(4,), dtype=tf.float64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [21]:
# Before creating our batches we need to add a property to our tensor dataset
# the ability to shuffle th observations every time we use this object
# the argument inside shuffle "buffer_size" gives you the amount of samples we wish to select
# after each shuffle, if buffer_size is greater than the number of elements inside
# the dataset it will simply take them all (it does not oversample)
train_shuffle = train.shuffle(buffer_size=len(X_train))
test_shuffle = test.shuffle(buffer_size=len(X_test))
# the shuffle method will give this property to the tensor dataset
for x, y in train_shuffle.take(1): 
  print('x:',x)
  print('y:',y)
for x, y in test_shuffle.take(1): 
  print('x:',x)
  print('y:',y)
# and now every time I use .take I get a different tensor
# same thing goes for next(iter())
x, y = next(iter(train_shuffle)) 
print('x:',x)
print('y:',y)
x, y = next(iter(test_shuffle)) 
print('x:',x)
print('y:',y)

x: tf.Tensor([5.5 4.2 1.4 0.2], shape=(4,), dtype=float64)
y: tf.Tensor(0, shape=(), dtype=int64)
x: tf.Tensor([5.5 2.4 3.7 1. ], shape=(4,), dtype=float64)
y: tf.Tensor(1, shape=(), dtype=int64)
x: tf.Tensor([4.8 3.1 1.6 0.2], shape=(4,), dtype=float64)
y: tf.Tensor(0, shape=(), dtype=int64)
x: tf.Tensor([6.3 2.8 5.1 1.5], shape=(4,), dtype=float64)
y: tf.Tensor(2, shape=(), dtype=int64)


In [22]:
# Now we are ready to form our batches, let's use the .batch method
train_batch = train_shuffle.batch(batch_size=8)
test_batch = test_shuffle.batch(batch_size=8)

# When extracting data from these objects we now get batches!
for x, y in train_batch.take(1): 
  print('x:',x)
  print('y:',y)
# This gives us a batch of 8 observations from the training data of 
# shape (8,4) (batch_size, ncol) and (8,) for the target associated with each
# observation in the batch

x: tf.Tensor(
[[5.1 3.5 1.4 0.3]
 [5.2 3.4 1.4 0.2]
 [6.  2.2 5.  1.5]
 [6.1 2.9 4.7 1.4]
 [5.8 4.  1.2 0.2]
 [6.5 2.8 4.6 1.5]
 [7.4 2.8 6.1 1.9]
 [6.5 3.  5.2 2. ]], shape=(8, 4), dtype=float64)
y: tf.Tensor([0 0 2 1 0 1 2 2], shape=(8,), dtype=int64)


2024-12-15 18:26:03.273320: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
